# How Packing Affects Timing

## Introduction
In this notebook we are going to see how different packing options affect the performance of our code. 
In this notebook we consider these 3 problems:
* Multiply a batch of vectors $x_1, \ldots, x_n$ by a single vector $w$ (the example from the previous notebook). 
* Multiply a batch of $n$ vector pairs $\langle v_1, u_1\rangle, \ldots, \langle v_n, u_n\rangle$. 
* Execute a sequence of 4 matrix multiplications: $M_1 \cdot M_2 \cdot M_3 \cdot M_4$
</ul>

For each of these problems we will measure:
<ul>
    <li> <i>Latency</i> - The time until receiving a first reply of a batch.</li>
    <li> <i>Amortized latency</i> - The amortized time to process a single query in a batch. </li>
    <li> <i>Memory</i> - The amount of memory needed to process the batch. </li>
</ul>

We start by importing the required packages for this tutorial.

In [ ]:
import math
import numpy as np

# For ploting graphs
from matplotlib import pyplot as plt

# Progress bar
from tqdm import tqdm

# For the time/memory measurements
from timeit import default_timer

from psutil import Process
# Utility method for checking memory
def get_used_ram(): 
    mem_MB = Process().memory_info().rss / (1024 * 1024)
    return mem_MB

## Step 1. Initializing

We first import Helayers and initialize it. This time we use a real CKKS context, implemented over [Microsoft SEAL](https://github.com/microsoft/SEAL)$^1$ to measure real timing and memory usage. We intialize the context to operate over ciphertexts of $2^{14}=16,384$ slots with 128-bits security. Finally, we create an `encoder` that we will later use for encoding and encrypting data.

$^1$ HeLayers supports a variety of libraries such as [HELib](https://homenc.github.io/HElib/), [Microsoft SEAL](https://github.com/microsoft/SEAL), [Palisade](https://palisade-crypto.org/), [HEAAN](https://heaan.it/), where we arbitrarily chose SEAL for this demonstration.

In [ ]:
import pyhelayers

print("Imported pyhelayers", pyhelayers.VERSION)

# Initialize a CKKS context with 16,384 slots.
requirement = pyhelayers.HeConfigRequirement(
    num_slots = 16*1024,            # Number of slots per ciphertext
    multiplication_depth = 8,       # Allow 8 levels of multiplications
    fractional_part_precision = 40, # Set the precision to 1/2^40.
    integer_part_precision = 20,    # Set the largest number to 2^20.
    security_level = 128)

# Use an HE context that uses Micorsoft SEAL
he_context = pyhelayers.SealCkksContext()
he_context.init(requirement)
print("Initialized a context with ", he_context.slot_count(), " slots")

# Create the Encoder using the context.
encoder = pyhelayers.TTEncoder(he_context)

<br>

## Example 1: Inner product of vectors with a constant vector.

In [the CTileTensor_basics tutorial](01_CTileTensor_basics.ipynb), we implemented a linear regression by implementing a dot product operation between two vectors $w, x \in \mathbb{R}^d$, where $w$ is the model weights vector and $x$ is the sample on which we performed the inference operation. In many cases, we would like to infer data on a batch of $n$ samples $x_1, \ldots, x_n \in \mathbb{R}^d$, where we compute the $n$ results $y_1, \ldots, y_n \in \mathbb{R}$ ,with $y_i = \langle x_i, w \rangle$.

We start by defining two utility functions `print_stats` and `genDemoInput`.

In [ ]:
def print_stats(s):
    """ Reports measurements parameters
        Input: a list of 4 elements s = [latency, amortized latency, total_ram, total_tiles_size], where
                 latency          - the latency of some computations,
                 amortized latency- the amortized latency of the computations, 
                 total_ram        - the total used RAM of the current process, 
                 total_tiles_size - the total tiles tensors memory size
    """
    print("Latency = ", s[0], " seconds.")
    print("Amortized latency = ", s[1], " seconds per input.")
    print("RAM of tile tensors = ", (s[3]/1024/1024), "MB.")

def genDemoInput(m, n, t=None):
    """ Generates an m x n matrix with values [1 ,..., m x n] or an m x n x t tensor with values in [1 ,..., m x n x t] when t is not None."""
    if t is not None:
        return np.arange(1, m*n*t + 1).reshape((m, n, t))
    else:
        return np.arange(1, m*n + 1).reshape((m, n))

Subsequently, we define the function `genAndEncData`, that gets the dimensions $n$ and $d$, in addition to another parameter $\alpha$ that controls the packing $0 \le \alpha \le log_2 (slotNum)$. This function, generates $w$ and $X_{n \times d}=[x_1,\ldots,x_n]$, pack them by broadcasting $w$ over the 0 axis $n$ times using the function `get_with_duplicated_dim(0)` and returns their encryption. 

In [ ]:
def genAndEncData(he_context, encoder, n, d, alpha):
    """ A packing-oblivious function that generates and encrypts the input (X) and the model weights (w).
        Input:   n     - number of x input vectors
                 d     - dimension of every vector
                 alpha - a parameter that controls the packing, 0 <= alpha <= log_2 (slotNum)
        
        Returns: cW        - an encrypted tile tensor of w
                 cX        - a vector of encrypted tile tensor that represent X
                 batch_num - number of batches
    """
    assert isinstance(alpha, int),                        "alpha must be integer"
    assert alpha >= 0,                                    "alpha must be non-negative"
    assert alpha <= math.log(he_context.slot_count(), 2), "alpha must be smaller than log_2 n"

    # Generate the input (plaintext) X = [x_1, ... , x_n] and the weight vector w
    X = genDemoInput(n, d)
    w = genDemoInput(1, d)

    # Compute the tiles shape based on alpha 
    nDim = pow(2, alpha)
    dDim = he_context.slot_count() / nDim
    batch_num = math.ceil(n/nDim)

    # Set the TT dimensions of X and w, where we use the function get_with_duplicated_dim() to broadcast w over the 0 axis n times.
    shapeX = pyhelayers.TTShape([nDim, dDim])
    shapeW = pyhelayers.TTShape([nDim, dDim]).get_with_duplicated_dim(0)
    
    # Encode and encrypt w and X = [x1, ..., xn]
    cW = encoder.encode_encrypt(shapeW, w)
    cX = [encoder.encode_encrypt(shapeX, X[i*nDim : (i+1)*nDim]) for i in range(batch_num)]
    
    return cW, cX, batch_num

Finally, the function `benchmarkLinearRegression` calls `genAndEncData` to generate an encrypted data and benchmark the linear regression computations. 

In [ ]:
def benchmarkLinearRegression(he_context, encoder, n, d, alpha):
    """ A packing-oblivious function that generates the input (X) and the model weights (w) in plaintext,
        subsequently, it iteratively encrypts batches of X and infer data from them using a linear regression model.
        Input:   n     - number of x inputs
                 d     - dimension of every vector
                 alpha - a parameter that controls the packing, 0 <= alpha <= log_2 (slotNum)
        
        Returns: the following measurements data: 
                 latency          - the latency of the computations,
                 Amortized latency- the amortized latency of the computations, 
                 total_ram        - the total used RAM of the current process, 
                 total_tiles_size - the total tiles tensors memory size
    """

    # Generate and Encrypt the data
    cW, cX, batch_num = genAndEncData(he_context, encoder, n, d, alpha)

    # Estimate the memory requirements of the tile tensors. 
    # Note that we take the maximum over the elements of X because in a real use-case every batch will be encrypted and evaluated separately
    # Thus the pick memory equals only the maximal tile tensor size.
    estMemW = cW.get_estimated_memory_usage_bytes()
    estMemX = max([cX[i].get_estimated_memory_usage_bytes() for i in range(batch_num)])

    # Initialize measuring parameters
    estMemOutput = 0
    startTime = default_timer()

    # Compute cOutput = X[i]*w for all batches and measure the latency/throuput
    for i in range(batch_num):
        cOutput = cX[i]
        cOutput.multiply(cW)
        
        # We measure the output memory usage here because it is reduced after calling get_sum_over_dim.
        estMemOutput = max(estMemOutput, cOutput.get_estimated_memory_usage_bytes())

        cOutput = cOutput.get_sum_over_dim(1)

    # Compute the final results.
    raw_time         = default_timer() - startTime
    latency          = raw_time / batch_num
    amortized_latency= raw_time/n
    total_ram        = get_used_ram()
    total_tiles_size = estMemW + estMemX + estMemOutput
    
    return latency, amortized_latency, total_ram, total_tiles_size

We first set $n=10{,}000$, $d=100$ and $\alpha=10$

In [ ]:
print_stats(benchmarkLinearRegression(he_context, encoder, 10*1000, 100, 10))

Next we note what happens to the runtime and memory when we use $\alpha=8$ instead of $10$.

In [ ]:
print_stats(benchmarkLinearRegression(he_context, encoder, 10*1000, 100, 8))

### Exercise

You are required to have latency of at most 0.2 seconds. What is the lowest amortized latency you can achieve?

## Example 2: Inner products
In this example we get as input 2 lists of $d$-dimensional vectors $u_1, \ldots, u_n$ and $v_1, \ldots, v_n$ and compute the dot-product $\langle u_i, v_i \rangle$. As before we start by defining the function `genAndEncData2`

In [ ]:
def genAndEncData2(he_context, encoder, n, d, alpha):
    """ A packing-oblivious function that generates and encrypts the input (X) and the model weights (w).
        Input:   n     - number of x inputs
                 d     - dimension of every vector
                 alpha - a parameter that controls the packing, 0 <= alpha <= log_2 (slotNum)
        
        Returns: cV        - a vector of encrypted tile tensor that represent V
                 cU        - a vector of encrypted tile tensor that represent U
                 batch_num - number of batches
    """
    assert isinstance(alpha, int),                        "alpha must be integer"
    assert alpha >= 0,                                    "alpha must be non-negative"
    assert alpha <= math.log(he_context.slot_count(), 2), "alpha must be smaller than log_2 n"

    nDim = pow(2, alpha)
    dDim = he_context.slot_count() / nDim
    batch_num = math.ceil(n/nDim)
    
    # Note that V and U has the same shape
    shapeInput = pyhelayers.TTShape([nDim, dDim])

    # Generate the input V = [v_1, ..., v_n] and U = [u_1, ..., u_n] (plaintext)
    pV = genDemoInput(n,d)
    pU = genDemoInput(n,d)

    # Encode and encrypt the data
    cV = [encoder.encode_encrypt(shapeInput, pV[i*nDim : (i+1)*nDim]) for i in range(batch_num)]
    cU = [encoder.encode_encrypt(shapeInput, pU[i*nDim : (i+1)*nDim]) for i in range(batch_num)]

    return cV, cU, batch_num

Finally, the function `benchmarkInnerProduct` calls `genAndEncData2` to generate an encrypted data and benchmark the linear regression computations. 

In [ ]:
def benchmarkInnerProduct(he_context, encoder, n, d, alpha):
    """ A packing-oblivious function that generates the input (X) and the model weights (w) in plaintext,
        subsequently, it iteratively encrypts batches of X and infer data from them using a linear regression model.
        Input:   n     - number of x inputs
                 d     - dimension of every vector
                 alpha - a parameter that controls the packing, 0 <= alpha <= log_2 (slotNum)
        
        Returns: the following measurements data: 
                 latency          - the latency of the computations,
                 amortized latency- the amortized latency of the computations, 
                 total_ram        - the total used RAM of the current process, 
                 total_tiles_size - the total tiles tensors memory size
    """

    cV, cU, batch_num = genAndEncData2(he_context, encoder, n, d, alpha)

    # Estimate the memory requirements of the tile tensors
    # Note that we take the maximum over the elements of U,V because in a real use-case every batch will be encrypted and evaluated separately
    # Thus the pick memory equals only the maximal tile tensor size.
    estMemV = max([cV[i].get_estimated_memory_usage_bytes() for i in range(batch_num)])
    estMemU = max([cU[i].get_estimated_memory_usage_bytes() for i in range(batch_num)])

    estMemOutput = 0
    startTime = default_timer()
    #print(cU[0].get_shape())
    # Compute cOutput = V[i]*U[i] for all batches and measure the latency/throuput
    for i in range(batch_num):
        cOutput = cU[i]
        cOutput.multiply(cV[i])
        
        # We measure the output memory usage here because it is reduced after calling get_sum_over_dim.
        estMemOutput = max(estMemOutput, cOutput.get_estimated_memory_usage_bytes())

        cOutput = cOutput.get_sum_over_dim(1)

    # Compute the final results.
    raw_time         = default_timer() - startTime
    latency          = raw_time/batch_num
    amortized_latency= raw_time/n
    total_ram        = get_used_ram()
    total_tiles_size = estMemU + estMemV + estMemOutput

    return latency, amortized_latency, total_ram, total_tiles_size

In [ ]:
print_stats(benchmarkInnerProduct(he_context, encoder, 10*1000, 100, 10))

Let's plot how the dependency between latency and amortized latency changes for different $\alpha$ values. For that we first benchmark `benchmarkInnerProduct` on different values of `alpha`.

In [ ]:
alphaRange = range(0,15)
d = 32

counts = np.zeros(shape = [0,4])
for alpha in tqdm(alphaRange):
    n=pow(2,alpha)
    counts = np.append(counts, [benchmarkInnerProduct(he_context, encoder, n,d,alpha)], axis=0)


Subsequently we use the function `plotAlphaDependntGraphs` that gets as input the range of $\alpha$ values and the measurements results and generates 3 plots for latency, amortized latency, and memory.

In [ ]:
def plotAlphaDependntGraphs(alphaRange, counts):
    latency, amortized_latency, ram, mem = counts.T
    
    # Plot a graph between alpha and the latency
    plt.plot(alphaRange, latency)
    plt.xlabel("alpha")
    plt.ylabel("latency (sec.)")
    plt.show()
    
    # Plot a graph between alpha and the latency
    plt.plot(alphaRange, amortized_latency)
    plt.xlabel("alpha")
    plt.ylabel("Amortized latency (sec. per input)")
    plt.show()
    
    # Plot a graph between alpha and the memory
    plt.plot(alphaRange, mem)
    plt.xlabel("alpha")
    plt.ylabel("Ram (bytes)")
    plt.show()

plotAlphaDependntGraphs(alphaRange, counts)

To explain the graphs, note that when $\alpha=14$ we pack slotCount pairs in our tile tensor, where the $i$-th pair is encoded in the $i$-th slot of the tensors. In this case when we run the inner-product code we do not use rotations. On the other extreme, when $\alpha=0$, we pack one pair. The tuples of each vector in the pair are mapped to different slots in ciphertexts.
For intermediate values $(0 < alpha < 14$) we map several tuples of each pair to different slots of the same ciphertext.

#### Explaining the latency graph

As expected, at both extremes ($\alpha=0$ and $\alpha=14$) the graph has local maxima.
When $\alpha=0$ the execution takes a long time because the inner-product requires rotations to sum over the slots of the product of $v$ and $u$.
When $\alpha=14$, indeed we do not require rotations but we operate over slotCount pairs.

#### Explaining the amortized latency graph

The amortized time to compute the inner-product reduces as alpha grows. This is expected since as alpha grows we need to perform less rotations, while the amortized number of other gates does not grow.

#### Explaining the memory graph

The memory needed to compute the inner product grows as alpha grows. This is expected since with higher values of alpha we keep more pairs in memory in each iteration to operate on.


## Automatically finding the best packing

As you saw in the exercise, finding the best (supported) packing given constrains is a repetitious boring task. The first improvement that comes to mind is writing a script containing a for-loop. When the shape of the TileTensor has more dimensions, naively brute-forcing over the solution space becomes infeasible. To avoid this complexity, HELayers includes a packing-optimizer that:

<ul>
    <li>supports constrains</li>
    <li>supports brute-force and also other heuristics such as gradient descent to find a good solution</li>
    <li>supports running in simulation mode where we run the code over plaintext and count gates to estimate the running time for each packing-option</li>
</ul>


## Example 3: Matrix-matrix multiplication

In this example, we show how to multiply 4 matrices: $M_1 \cdot M_2 \cdot M_3 \cdot M_4$,
where the dimensions are as follows:
<ul>
    <li>$M_1 \in \mathbb{R}^{a \times b}$</li>
    <li>$M_2 \in \mathbb{R}^{b \times c}$</li>
    <li>$M_3 \in \mathbb{R}^{c \times d}$</li>
    <li>$M_4 \in \mathbb{R}^{d \times e}$</li>
</ul>

To do that efficiently we will introduce a 3rd dimension which is going to be duplicated. The index of the duplicated dimension is different for each matrix. Specifically, we are going to pack the matrices this way:
<ul>
    <li>$M_1$ is packed as $[b,a,*]$</li>
    <li>$M_2$ is packed as $[b,*,c]$</li>
    <li>$M_3$ is packed as $[d,*,c]$</li>
    <li>$M_4$ is packed as $[d,*,e]$</li>
</ul>

We pack each matrix as a 3-dimensional tensor. The choice of shapes depend on the matrices we multiply. For example, when multiplying $M_1 \cdot M_2$:
<ul>
    <li>We set the mutual matrix dimension ($b$) at the same tensor dimension.</li>
    <li>The other matrix dimensions ($a$ and $c$) are placed at different tensor dimensions</li>
    <li>Tensor dimension that no matrix dimension was mapped to are made duplicated.</li>
    <li>The encoding of $M_1$ seems "transposed". This is because placing the mutual matrix dimension ($b$) at the first tensor has advantages.</li>
</ul>
The shapes of $M_3$ and $M_4$ are similarly chosen to match the shape of the output $(M_1 \cdot M_2)$ and $((M_1 \cdot M_2) \cdot M_3)$.
  

The result of multiplying $M_1 \cdot M_2$ is $[b,a,c]$ and after summing over the 1st tensor dimension we get $[*,a,c]$. Multipling it with $M_3$ (whose shape is $[d,*,c]$) we get a tensor whose shape is $[d,a,c]$. After summing over the 3rd tensor dimension we get $[d,a,1?]$ and after broadcasting we get $[d,a,*]$.
Multiplying by $M_4$ (whose shape is $[d,*,e]$) we get a tensor whose shape is $[d,a,e]$. After summing over the 1st tensor dimension we get $[*,a,e]$.

Note that when summing over the 1st tensor dimension the output is already duplicated, which is why we prefer to encode $M_1$ with a shape that seems transposed.


In [ ]:
def benchmark_matrix_matrix_multiplication(a, b, c, d, e, alpha, beta):
    assert isinstance(alpha, int) and isinstance(beta, int), "alpha and beta must be integer"
    assert alpha >= 0 and beta >= 0, "alpha and beta must be non-negative"
    assert alpha+beta <= math.log(he_context.slot_count(), 2), "alpha+beta must be smaller than log_2 n"

    t1 = pow(2, alpha)
    t2 = pow(2, beta)
    t3 = he_context.slot_count() / t1 / t2

    shapeFirst = pyhelayers.TTShape([t1, t2, t3]).get_with_duplicated_dim(2)
    shapeLater = pyhelayers.TTShape([t1, t2, t3]).get_with_duplicated_dim(1)
    
    # Generate the input matrix M1 (plaintext)
    pM1 = genDemoInput(b, a, 1)
    pM2 = genDemoInput(b, 1, c)
    pM3 = genDemoInput(d, 1, c)
    pM4 = genDemoInput(d, 1, e)

    # Encrypt pM1 - pM4
    cM1 = encoder.encode_encrypt(shapeFirst, pM1)
    cM2 = encoder.encode_encrypt(shapeLater, pM2)
    cM3 = encoder.encode_encrypt(shapeLater, pM3)
    cM4 = encoder.encode_encrypt(shapeLater, pM4)

    estMem = cM1.get_estimated_memory_usage_bytes() + \
             cM2.get_estimated_memory_usage_bytes() + \
             cM3.get_estimated_memory_usage_bytes() + \
             cM4.get_estimated_memory_usage_bytes()

    print(f"M1 shape = \t\t\t\t\t{cM1.get_shape()}")
    print(f"M2 shape = \t\t\t\t\t{cM2.get_shape()}")

    startTime = default_timer()
    
    cOut = cM1
    cOut.multiply(cM2)
    cOut = cOut.get_sum_over_dim(0)

    print(f"M1 * M2 shape = \t\t\t\t{cOut.get_shape()}\n")
    print(f"M3 shape = \t\t\t\t\t{cM3.get_shape()}")
    
    cOut.multiply(cM3)
    cOut = cOut.get_sum_over_dim(2)
    print(f"M1 * M2 * M3 shape = \t\t\t\t{cOut.get_shape()}")
    cOut.clear_unknowns()
    print(f"M1 * M2 * M3 shape (After clearing unknowns) = \t{cOut.get_shape()}")
    cOut.duplicate_over_dim(2)

    print(f"M1 * M2 * M3 shape (After duplicating) = \t{cOut.get_shape()}\n")
    print(f"M4 shape = \t\t\t\t\t{cM4.get_shape()}")
        
    cOut.multiply(cM4)
    cOut = cOut.get_sum_over_dim(0)
    
    print(f"M1 * M2 * M3 * M4 shape = \t\t\t{cOut.get_shape()}\n")
    
    # Assert the output contains the expected shape
    assert (cOut.get_shape().get_original_sizes()==[1,a,e]).all()

    latency = amortized_latency = (default_timer() - startTime)
    ram = get_used_ram()    
    return latency, amortized_latency, ram, estMem

In [ ]:
print_stats(benchmark_matrix_matrix_multiplication(4,5,6,7,8,4,4))


## Step 5. Up next

In this notebook we explored with various packing options, and saw how they affect performance. HELayers actually has an automatic optimizer, that finds the optimal packing for a given computation. 
You can see in the parent folder, demo 02, how to read a Keras model with HELayers, find an optimal packing for the input and run it over FHE (CKKS).

The next notebook in this tutorial shows how we can use tile tensors to simulate large ciphertexts with smaller ones, simplifying algorithm design and improve performance.
<br>